In [1]:
import pickle
import pandas as pd
import os
import sys
import gensim
import nltk
import re
sys.path.append('..')
from src.utils import load_catalog, pickle_partial
from sklearn.model_selection import train_test_split

import logging
logging.getLogger().setLevel('INFO')

### Create a dataframe containing for each book id if the book covers the genres of interest

Genres of interest

In [2]:
genres_mapping = {
    'short stories':[],
    'sci-fi':['science fiction'],
    'historical':['historical fiction'],
    'drama':[],
    'poetry':[],
    'adventure':['adventure stories'],
    'detective & mystery':['detective and mystery stories'],
    'love stories':[],
    'biography':[],
    'western':['western stories'],
    'children lit.':[
        "children", "children's stories", "fairy tales", "girls", "boys", "children literature"],
    'philosophy & ethics':['philosophy','ethics'],
    'fantasy lit.':['fantasy literature'],
    'religion & mythology':[
        'christian life', 'christianity', 'religion', 'mythology']
}

In [3]:
def simple_genres(subject_set):
    """
    Splits each LCSH subject on dash and comma
    """
    result = set()
    for s in subject_set:
        arr = s.replace('--',',').split(',')
        for a in arr:
            result.add(a.strip().lower())
    return result

Split the complex LCSH tags into simple ones.

In [4]:
metadata_catalog = load_catalog()
metadata_catalog['simple_genres'] = \
    metadata_catalog.subjects.apply(lambda x: simple_genres(x))

downloading catalog from https://drive.google.com/uc?export=download&id=1fX2I062qGzneJ1ysz_0hR_K831T6g0Gd


- Rows ~ book ids
- Columns ~ genres
- The cell in the dataframe is 1 if the book covers the genre (and 0 otherwise).

In [5]:
df_genres = pd.DataFrame(metadata_catalog['simple_genres'].copy())

for genre, other_genre_tags in genres_mapping.items():
    genre_tags = set([genre]+other_genre_tags)
    df_genres.loc[:,genre] = df_genres.simple_genres.apply(
        lambda x: 1 if len(genre_tags.intersection(x)) > 0 else 0)
df_genres.drop('simple_genres', axis=1, inplace=True)

# drop books which do not contain any genre of interest
df_genres = df_genres[df_genres.sum(axis=1)>=1]

pickle.dump(df_genres, open('../res/df_genres_all.pkl','wb'))

Books with exactly one of the specified genres

In [6]:
print(df_genres.shape)
df_genres_1 = df_genres[df_genres.sum(axis=1) == 1]
pickle.dump(df_genres_1, open('../res/df_genres_ex_1.pkl','wb'))
print(df_genres_1.shape)

(13562, 14)
(11861, 14)


# Create snippets of various lengths and tokenize them

**!** Here you need raw book texts from Project Gutenberg, feel free to download them.

If you don't have them, you can skip to the section - **Load and store texts with length 200**

In [8]:
books_dir = '../res/books_clean/'
snippets_dir_template = '../res/documents'

doc_length_in_chars = [200, 800, 3200]
n_samples_per_book = 50
skip_first_n_chars = 10000
skip_last_n_chars = 10000

In [9]:
def preprocess_text(text):
    """
    Does minor preprocessing on the text.
    """
    # remove stars
    endline_wrap = re.compile(r"-\n")
    filter_regex = re.compile(r"[_*]")
    # substitute quotation marks
    double_quot_regex = re.compile(r"[“”]")
    single_quot_regex = re.compile(r"[’‘]")
    # replace whitespace sequence by single space
    whitespace_sequence = re.compile(r"\s+")
    
    text = endline_wrap.sub("", text)
    text = filter_regex.sub("", text)
    text = double_quot_regex.sub('"', text)
    text = single_quot_regex.sub("'", text)
    text = whitespace_sequence.sub(" ", text)
    return text

### Create snippets of length 200, 800 and 3200 characters

In [3]:
def create_books(input_dir,
                 output_dir,
                 doc_length_in_chars,
                 n_samples_per_book,
                 keep_ids = None,
                 skip_first_n_chars=0,
                 skip_last_n_chars=0):
    
    # create output dirs if not exists
    for doc_length in doc_length_in_chars:
        folder_name = '{}_{}'.format(output_dir, doc_length)
        os.makedirs(folder_name, exist_ok=True)
        
    max_doc_length = max(doc_length_in_chars)
    n_books = len(os.listdir(input_dir))
    i = 0
    for f in os.listdir(input_dir):
        i += 1
        try:
            book_id = int(f.split('_')[0].split('.')[0])
        except ValueError:
            print("{}/{}, did not open {}".format(i, n_books, f))
            continue
        
        # checks if it is a book of interest
        if keep_ids is not None and book_id not in keep_ids:
            continue
        print("{}/{}, book {}".format(i, n_books, f), end='')
        text = open(os.path.join(input_dir, f),'r',encoding="UTF-8").read()

        # clean text
        text = preprocess_text(text)
        # drop the heading and trailing characters as they might contain book contents or glossaries
        # which are not the typical book text
        text = text[skip_first_n_chars : -skip_last_n_chars]
        text_length_in_chars = len(text)

        max_distinct_docs = max(text_length_in_chars // max_doc_length, 0)
        n_docs = min(n_samples_per_book, max_distinct_docs)


        if n_docs > 1:
                space_size = (text_length_in_chars - n_docs * max_doc_length) // (n_docs-1)
        else:
            space_size = 0

        for doc_number in range(n_docs):
            # creating the largest of the document lengths
            start_index = doc_number * (space_size + max_doc_length)
            end_index = start_index + max_doc_length
            doc = text[start_index : end_index]

            for doc_length in doc_length_in_chars:
                # throw away first and last non-complete word
                d = doc[:doc_length][doc.find(' ') + 1 : doc.rfind(' ')]
                folder_name = '{}_{}'.format(output_dir, doc_length)
                file_name = '{:d}_{:d}.txt'.format(book_id, doc_number)
                filepath = os.path.join(folder_name, file_name) 
                with open(filepath, 'w', encoding='UTF-8') as outp_f:
                    outp_f.write(d)
        print(", {} parts created".format(n_docs))

In [239]:
%%time
create_books(input_dir=books_dir',
             output_dir=snippets_dir_template,
            doc_length_in_chars=doc_length_in_chars,
            n_samples_per_book=n_samples_per_book,
            skip_first_n_chars=skip_first_n_chars,
            skip_last_n_chars=skip_last_n_chars)

1/17214, book 38254, 50 parts created
2/17214, book 1069, 50 parts created
3/17214, book 21379, 50 parts created
4/17214, book 1855, 12 parts created
5/17214, book 34499, 50 parts created
6/17214, book 12369, 50 parts created
7/17214, book 19781, 50 parts created
8/17214, book 17429, 24 parts created
9/17214, book 29326, 8 parts created
10/17214, book 35779, 48 parts created
11/17214, book 13089, 50 parts created
12/17214, book 18461, 50 parts created
13/17214, book 4729, 50 parts created
14/17214, book 40940, 50 parts created
15/17214, book 29742, 4 parts created
16/17214, book 3954, 19 parts created
17/17214, book 18839, 39 parts created
18/17214, book 2886, 50 parts created
19/17214, book 23278, 50 parts created
20/17214, book 3168, 24 parts created
21/17214, book 40518, 50 parts created
22/17214, book 44984, 50 parts created
23/17214, book 10268, 50 parts created
24/17214, book 32904, 5 parts created
25/17214, book 11188, 0 parts created
26/17214, book 15528, 19 parts created
27/17

213/17214, book 14436, 50 parts created
214/17214, book 11978, 50 parts created
215/17214, book 22530, 50 parts created
216/17214, book 46075, 50 parts created
217/17214, book 35587, 50 parts created
218/17214, book 13277, 50 parts created
219/17214, book 2620, 50 parts created
220/17214, book 26190, 0 parts created
221/17214, book 11520, 50 parts created
222/17214, book 33670, 50 parts created
223/17214, book 36302, 50 parts created
224/17214, book 15180, 50 parts created
225/17214, book 1899, 50 parts created
226/17214, book 14052, 50 parts created
227/17214, book 39178, 0 parts created
228/17214, book 32590, 2 parts created
229/17214, book 41062, 7 parts created
230/17214, book 20869, 50 parts created
231/17214, book 39944, 50 parts created
232/17214, book 21187, 50 parts created
233/17214, book 36130, 50 parts created
234/17214, book 38896, 50 parts created
235/17214, book 13879, 19 parts created
236/17214, book 34667, 50 parts created
237/17214, book 46247, 50 parts created
238/17

628/17214, book 22767, 50 parts created
629/17214, book 14498, 50 parts created
630/17214, book 10804, 50 parts created
631/17214, book 38652, 50 parts created
632/17214, book 10038, 50 parts created
633/17214, book 391, 49 parts created
634/17214, book 19387, 24 parts created
635/17214, book 23028, 27 parts created
636/17214, book 29512, 50 parts created
637/17214, book 26768, 50 parts created
638/17214, book 29720, 0 parts created
639/17214, book 3936, 28 parts created
640/17214, book 15778, 50 parts created
641/17214, book 38460, 50 parts created
642/17214, book 12139, 50 parts created
643/17214, book 30269, 50 parts created
644/17214, book 40922, 50 parts created
645/17214, book 38838, 50 parts created
646/17214, book 21129, 50 parts created
647/17214, book 1239, 50 parts created
648/17214, book 25589, 18 parts created
649/17214, book 1837, 50 parts created
650/17214, book 38236, 50 parts created
651/17214, book 24669, 0 parts created
652/17214, book 31189, 50 parts created
653/172

1042/17214, book 18297, 50 parts created
1043/17214, book 34861, 50 parts created
1044/17214, book 353, 42 parts created
1045/17214, book 15588, 50 parts created
1046/17214, book 11128, 50 parts created
1047/17214, book 41658, 50 parts created
1048/17214, book 44118, 50 parts created
1049/17214, book 14668, 22 parts created
1050/17214, book 33876, 50 parts created
1051/17214, book 36538, 50 parts created
1052/17214, book 44924, 50 parts created
1053/17214, book 39570, 50 parts created
1054/17214, book 32198, 50 parts created
1055/17214, book 19177, 0 parts created
1056/17214, book 161, 50 parts created
1057/17214, book 27678, 50 parts created
1058/17214, book 2826, 22 parts created
1059/17214, book 39114, 50 parts created
1060/17214, book 24499, 50 parts created
1061/17214, book 13029, 50 parts created
1062/17214, book 13815, 50 parts created
1063/17214, book 29386, 50 parts created
1064/17214, book 19721, 50 parts created
1065/17214, book 25779, 50 parts created
1066/17214, book 39326

1245/17214, book 39726, 50 parts created
1246/17214, book 31913, 7 parts created
1247/17214, book 22538, 11 parts created
1248/17214, book 1891, 7 parts created
1249/17214, book 15188, 50 parts created
1250/17214, book 14268, 44 parts created
1251/17214, book 44518, 21 parts created
1252/17214, book 36138, 50 parts created
1253/17214, book 36904, 50 parts created
1254/17214, book 39170, 50 parts created
1255/17214, book 15527, 50 parts created
1256/17214, book 21712, 50 parts created
1257/17214, book 44385, 50 parts created
1258/17214, book 10869, 50 parts created
1259/17214, book 11187, 0 parts created
1260/17214, book 40725, 50 parts created
1261/17214, book 1402, 50 parts created
1262/17214, book 2287, 50 parts created
1263/17214, book 30452, 50 parts created
1264/17214, book 18238, 8 parts created
1265/17214, book 17270, 9 parts created
1266/17214, book 23045, 47 parts created
1267/17214, book 22197, 5 parts created
1268/17214, book 3167, 20 parts created
1269/17214, book 45859, 50

...

16076/17214, book 2492, 50 parts created
16077/17214, book 30247, 50 parts created
16078/17214, book 42667, 0 parts created
16079/17214, book 26322, 50 parts created
16080/17214, book 1217, 50 parts created
16081/17214, book 40130, 4 parts created
16082/17214, book 21107, 50 parts created
16083/17214, book 38816, 50 parts created
16084/17214, book 37062, 48 parts created
16085/17214, book 44590, 50 parts created
16086/17214, book 41224, 46 parts created
16087/17214, book 33604, 50 parts created
16088/17214, book 45684, 50 parts created
16089/17214, book 20213, 22 parts created
16090/17214, book 46001, 17 parts created
16091/17214, book 16771, 50 parts created
16092/17214, book 22544, 8 parts created
16093/17214, book 13203, 7 parts created
16094/17214, book 19739, 50 parts created
16095/17214, book 31153, 5 parts created
16096/17214, book 3786, 50 parts created
16097/17214, book 29990, 2 parts created
16098/17214, book 34613, 50 parts created
16099/17214, book 46233, 50 parts created
1

16469/17214, book 33232, 50 parts created
16470/17214, book 24285, 50 parts created
16471/17214, book 11162, 0 parts created
16472/17214, book 25357, 32 parts created
16473/17214, book 39708, 50 parts created
16474/17214, book 14410, 13 parts created
16475/17214, book 36740, 50 parts created
16476/17214, book 41420, 50 parts created
16477/17214, book 25165, 11 parts created
16478/17214, book 44152, 0 parts created
16479/17214, book 25959, 49 parts created
16480/17214, book 46605, 50 parts created
16481/17214, book 19901, 50 parts created
16482/17214, book 23292, 50 parts created
16483/17214, book 16949, 50 parts created
16484/17214, book 3182, 22 parts created
16485/17214, book 41044, 50 parts created
16486/17214, book 40196, 50 parts created
16487/17214, book 39962, 50 parts created
16488/17214, book 14246, 35 parts created
16489/17214, book 35793, 50 parts created
16490/17214, book 19559, 50 parts created
16491/17214, book 13251, 50 parts created
16492/17214, book 31101, 50 parts cre

16668/17214, book 38286, 50 parts created
16669/17214, book 27860, 50 parts created
16670/17214, book 35599, 50 parts created
16671/17214, book 18681, 33 parts created
16672/17214, book 19753, 50 parts created
16673/17214, book 31139, 50 parts created
16674/17214, book 29300, 50 parts created
16675/17214, book 12941, 50 parts created
16676/17214, book 26348, 50 parts created
16677/17214, book 19595, 43 parts created
16678/17214, book 39192, 50 parts created
16679/17214, book 37834, 50 parts created
16680/17214, book 15956, 50 parts created
16681/17214, book 32748, 9 parts created
16682/17214, book 10418, 50 parts created
16683/17214, book 40368, 50 parts created
16684/17214, book 1873, 34 parts created
16685/17214, book 30823, 50 parts created
16686/17214, book 23408, 5 parts created
16687/17214, book 27894, 33 parts created
16688/17214, book 29132, 2 parts created
16689/17214, book 3718, 7 parts created
16690/17214, book 38616, 2 parts created
16691/17214, book 35109, 50 parts created

16865/17214, book 33857, 50 parts created
16866/17214, book 32985, 50 parts created
16867/17214, book 140, 50 parts created
16868/17214, book 2807, 50 parts created
16869/17214, book 21333, 50 parts created
16870/17214, book 36384, 50 parts created
16871/17214, book 37256, 34 parts created
16872/17214, book 32724, 2 parts created
16873/17214, book 24641, 8 parts created
16874/17214, book 10474, 50 parts created
16875/17214, book 1023, 50 parts created
16876/17214, book 18619, 50 parts created
16877/17214, book 29962, 0 parts created
16878/17214, book 23464, 50 parts created
16879/17214, book 31393, 50 parts created
16880/17214, book 26324, 50 parts created
16881/17214, book 35733, 50 parts created
16882/17214, book 21101, 50 parts created
16883/17214, book 32516, 50 parts created
16884/17214, book 40136, 50 parts created
16885/17214, book 1211, 50 parts created
16886/17214, book 24473, 7 parts created
16887/17214, book 37858, 50 parts created
16888/17214, book 42205, 50 parts created
1

17066/17214, book 20475, 50 parts created
17067/17214, book 14640, 5 parts created
17068/17214, book 25107, 0 parts created
17069/17214, book 39558, 50 parts created
17070/17214, book 33062, 50 parts created
17071/17214, book 149, 50 parts created
17072/17214, book 27650, 36 parts created
17073/17214, book 31735, 7 parts created
17074/17214, book 26782, 18 parts created
17075/17214, book 34247, 50 parts created
17076/17214, book 22322, 17 parts created
17077/17214, book 19963, 4 parts created
17078/17214, book 43127, 50 parts created
17079/17214, book 27462, 6 parts created
17080/17214, book 31507, 50 parts created
17081/17214, book 22110, 0 parts created
17082/17214, book 21795, 39 parts created
17083/17214, book 44302, 50 parts created
17084/17214, book 32382, 50 parts created
17085/17214, book 25769, 50 parts created
17086/17214, book 19731, 50 parts created
17087/17214, book 30089, 50 parts created
17088/17214, book 31967, 11 parts created
17089/17214, book 28076, 0 parts created
1

## Load and store text snippets with various length

Function to load text snippets, tokenize them (with and without punctuation)

In [3]:
def load_docs(input_dir, keep_ids=None):
    
    simple_tokenized_texts = []
    word_tokenize_texts = []
    ids = []
    n_docs = len(os.listdir(input_dir))
    if keep_ids:
        n_docs = min(n_docs, len(keep_ids))
    i = 0
    for f in os.listdir(input_dir):
        
        # checks if it is a book of interest
        if keep_ids is not None and f.split('.')[0] not in keep_ids:
            continue

        # skip hidden files starting with dot
        if f[0] == '.':
            logging.info("{}/{}, skipping {}".format(i, n_docs, f))
            continue
        # show progress every 10000 books processed
        i += 1
        if i % 10000 == 0:
            print("{}/{}".format(i, n_docs))
        text = open(os.path.join(input_dir, f),'r',encoding="UTF-8").read().lower()
        # simple tokenizer (ignores punctuation and digits)
        simple_tokenized_text = list(gensim.utils.simple_tokenize(text))
        simple_tokenized_texts.append(simple_tokenized_text)
        # word_tokenize
        word_tokenize_text = nltk.word_tokenize(text)
        word_tokenize_texts.append(word_tokenize_text)
        
        ids.append(f.split('.')[0])
    return simple_tokenized_texts, word_tokenize_texts, ids

### Load and store texts with length 200

The contents of the `'../res/documents_200'` is here in a zipped format, download and extract to continue: `https://drive.google.com/uc?export=download&id=1buEQRtORg01bGzw05g03LPSdLtyHx67U`

In [9]:
%%time 
simple_tokenized_texts_200, word_tokenize_text_200, ids = load_docs('../res/documents_200/')

CPU times: user 1.55 s, sys: 157 ms, total: 1.71 s
Wall time: 1.88 s


In [10]:
pickle.dump(simple_tokenized_texts_200, open('../res/documents_200_simple_tokens.pkl','wb'))
pickle.dump(word_tokenize_text_200, open('../res/documents_200_word_tokenize_tokens.pkl','wb'))
pickle.dump(ids, open('../res/documents_ids.pkl','wb'))

### Load and store texts with length 800

In [11]:
%%time 
simple_tokenized_texts_800, word_tokenize_text_800, ids = load_docs('../res/documents_800/')

DEBUG:root:10000/673549
DEBUG:root:20000/673549
DEBUG:root:30000/673549
DEBUG:root:40000/673549
DEBUG:root:50000/673549
DEBUG:root:60000/673549
DEBUG:root:70000/673549
DEBUG:root:80000/673549
DEBUG:root:90000/673549
DEBUG:root:100000/673549
DEBUG:root:110000/673549
DEBUG:root:120000/673549
DEBUG:root:130000/673549
DEBUG:root:140000/673549
DEBUG:root:150000/673549
DEBUG:root:160000/673549
DEBUG:root:170000/673549
DEBUG:root:180000/673549
DEBUG:root:190000/673549
DEBUG:root:200000/673549
DEBUG:root:210000/673549
DEBUG:root:220000/673549
DEBUG:root:230000/673549
DEBUG:root:240000/673549
DEBUG:root:250000/673549
DEBUG:root:260000/673549
DEBUG:root:270000/673549
DEBUG:root:280000/673549
DEBUG:root:290000/673549
DEBUG:root:300000/673549
DEBUG:root:310000/673549
DEBUG:root:320000/673549
DEBUG:root:330000/673549
DEBUG:root:340000/673549
DEBUG:root:350000/673549
DEBUG:root:360000/673549
DEBUG:root:370000/673549
DEBUG:root:380000/673549
DEBUG:root:390000/673549
DEBUG:root:400000/673549
DEBUG:roo

CPU times: user 20min 39s, sys: 3min 21s, total: 24min 1s
Wall time: 29min 36s


In [12]:
pickle.dump(simple_tokenized_texts_800, open('../res/documents_800_simple_tokens.pkl','wb'))
pickle.dump(word_tokenize_text_800, open('../res/documents_800_word_tokenize_tokens.pkl','wb'))

### Load and store texts with length 3200

In [ ]:
%%time 
simple_tokenized_texts_3200, word_tokenize_text_3200, ids = load_docs('../res/documents_3200/',
                                                                      keep_ids=set(genres.index))

20000/673549
70000/673549
90000/673549
150000/673549
260000/673549
270000/673549
300000/673549
310000/673549
330000/673549
350000/673549
360000/673549
400000/673549
410000/673549
430000/673549
440000/673549
450000/673549
470000/673549
490000/673549
500000/673549
590000/673549
630000/673549
640000/673549
CPU times: user 26min 25s, sys: 1min 48s, total: 28min 14s
Wall time: 29min 48s


The file is to big to pickle as one file, splitting into 2

In [ ]:
pickle_partial.dump(simple_tokenized_texts_3200, open('../res/documents_3200_simple_tokens.pkl','wb'), parts=2)
pickle_partial.dump(word_tokenize_text_3200, open('../res/documents_3200_word_tokenize_tokens.pkl','wb'), parts=2)

### Store train and test ids for genres

In [8]:
ids = pickle.load(open('../res/documents_ids.pkl','rb'))

Copy the genres entry for each snippet

In [9]:
doc_ids_int = [int(d.split('_')[0]) for d in ids]
genres = pd.DataFrame(df_genres.reindex(doc_ids_int).values,
                                columns=df_genres.columns,
                                index=ids)
genres = genres[genres.sum(axis=1) >= 1]
pickle.dump(genres, open('../res/genres_snippets.pkl','wb'))
print(genres.shape)

(250191, 14)


Snippets with only one genre

In [10]:
genres_1 = genres[genres.sum(axis=1) == 1]
pickle.dump(genres_1, open('../res/genres_snippets_ex_1.pkl','wb'))
print(genres_1.shape)

(225334, 14)


Fix the train and test set

In [12]:
train_ids_1_genre, test_ids_1_genre = train_test_split(list(genres_1.index), random_state=42, test_size=0.15)
ids_multiple_genres = list(genres[genres.sum(axis=1) > 1].index)
print(len(ids_multiple_genres))
print(len(train_ids_1_genre))
print(len(test_ids_1_genre))

pickle.dump(ids_multiple_genres, open('../res/ids_multiple_genres.pkl','wb'))
pickle.dump(train_ids_1_genre, open('../res/ids_train_1_v2_genre.pkl','wb'))
pickle.dump(test_ids_1_genre, open('../res/ids_test_1_v2_genre.pkl','wb'))

24857
191533
33801


In [13]:
df_genres

,short stories,sci-fi,historical,drama,poetry,adventure,detective & mystery,love stories,biography,western,children lit.,philosophy & ethics,fantasy lit.,religion & mythology
id,,,,,,,,,,,,,,
11,0,0,0,0,0,0,0,0,0,0,0,0,1,0
12,0,0,0,0,0,0,0,0,0,0,0,0,1,0
15,0,0,0,0,0,1,0,0,0,0,0,0,0,0
16,0,0,0,0,0,0,0,0,0,0,0,0,1,0
19,0,0,0,0,1,0,0,0,0,0,0,0,0,0
20,0,0,0,0,1,0,0,0,0,0,0,0,0,0
23,0,0,0,0,0,0,0,0,1,0,0,0,0,0
24,0,0,1,0,0,0,0,0,0,0,0,0,0,0
26,0,0,0,0,1,0,0,0,0,0,0,0,0,0
